In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#from mangaki.utils.xals import MangakiXALS
from mangaki.utils.als import MangakiALS
from mangaki.utils.svd import MangakiSVD

In [10]:
from mangaki.utils.algo import get_dataset_backup
from mangaki.utils.data import Dataset
from mangaki.utils.values import rating_values

dataset = Dataset()#get_dataset_backup('als')
dataset.load_csv('balse/ratings.csv', convert=lambda x: rating_values[x])

In [11]:
svd = MangakiSVD(20)
svd.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)

i_train, i_test = train_test_split(range(len(dataset.anonymized.X)), test_size=0.2)

In [12]:
X_train = dataset.anonymized.X[i_train]
y_train = dataset.anonymized.y[i_train]
X_test = dataset.anonymized.X[i_test]
y_test = dataset.anonymized.y[i_test]

In [13]:
#svd.fit(dataset.anonymized.X[i_train], dataset.anonymized.y[i_train])
svd.all_errors(X_train, X_test, y_train, y_test)

Computing M: (2079 × 9979)


/Users/jilljenn/code/mangaki/mangaki/mangaki/utils/svd.py:36: RuntimeWarning: invalid value encountered in double_scalars
  means[i] = np.sum(matrix[i]) / np.sum(matrix[i] != 0)


Chrono: fill and center matrix [0q, 2096ms]
Shapes (2079, 20) (20,) (20, 9979)
Chrono: factor matrix [0q, 762ms]
Train error 1.07384664884
Test error 1.18565986226


In [14]:
als = MangakiALS(10, LAMBDA=0.1)
als.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)

#als.fit(dataset.anonymized.X[i_train], dataset.anonymized.y[i_train])
als.all_errors(X_train, X_test, y_train, y_test)

Computing M: (2079 × 9979)
Chrono: fill and center matrix [0q, 1793ms]
Shapes (2079, 10) (10, 9979)
Chrono: factor matrix [0q, 6424ms]
Train error 0.970174012756
Test error 1.16396110461


In [15]:
y_train_pred = als.predict(X_train)

In [16]:
from scipy.sparse import coo_matrix

nb_users = dataset.anonymized.nb_users
nb_works = dataset.anonymized.nb_works
row = X_train[:, 0]
col = X_train[:, 1]
data = y_train - y_train_pred
M_res = coo_matrix((data, (row, col)), shape=(nb_users, nb_works)).tocsr()

In [17]:
user_ids = sorted(set(row))

In [18]:
from mangaki.utils.balse import MangakiBALSE
from scipy.sparse import load_npz

T = load_npz('../data/balse/tag-matrix.npz').toarray()

In [473]:
from collections import defaultdict

reg = defaultdict(lambda: Lasso(alpha=0.01, normalize=True))
for user_id in user_ids:
    indices = M_res[user_id].indices
    values = M_res[user_id].data
    reg[user_id].fit(T[indices], values)
    if user_id % 100 == 0:
        print(user_id)

0
100
200
300
400


/Users/jilljenn/code/mangaki/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1800
1900
2000


In [474]:
M_res.tocsr()[0, :]

<1x9979 sparse matrix of type '<class 'numpy.float64'>'
	with 78 stored elements in Compressed Sparse Row format>

In [475]:
from sklearn.linear_model import LinearRegression

In [476]:
#M_pred = T.dot(W.T).T
#M_pred[user_id][work_id]
y_delta_res = []
for user_id, work_id in X_tmp_test:
    if user_id not in reg:
        print(user_id, 'not')
        y_delta_res.append(0)
    else:
        y_delta_res.append(reg[user_id].predict(T[work_id].reshape(1, -1))[0])

742 not
1966 not
1040 not
1177 not
819 not
819 not
193 not
492 not
887 not
262 not
963 not
670 not
1700 not


In [508]:
y_delta_repaired = list(map(relu, y_delta_res))

In [509]:
print('Test error', als.compute_rmse(y_pred + y_delta_res, y_tmp_test))

Test error 1.65421868781


# LOL

In [27]:
y_pred = als.predict(X_tmp_test)
y_new_pred = lasso.predict(X_tmp_test)

890 not
588 not
1001 not
866 not
660 not
1793 not
866 not
706 not
1887 not
492 not
868 not
1392 not
521 not
521 not


In [74]:
WINDOW = 1000
print('Test error', als.compute_rmse(y_als[-WINDOW:], y_tmp_test[-WINDOW:]))

Test error 1.32958817247


In [75]:
y_new_pred = pred.eval()
print('Test error', als.compute_rmse(y_new_pred[-WINDOW:], y_tmp_test[-WINDOW:]))

Test error 1.26236346989


In [76]:
print('Test error', als.compute_rmse(y_lasso[-WINDOW:], y_tmp_test[-WINDOW:]))

Test error 1.2935313945


In [77]:
y_balse = balse.predict(X_tmp_test)
print('Test error', als.compute_rmse(y_balse[-WINDOW:], y_tmp_test[-WINDOW:]))

104 not
160 not
687 not
1263 not
159 not
1887 not
492 not
352 not
521 not
1460 not
521 not
491 not
352 not
1584 not
2002 not
Test error 1.2935313945


In [54]:
nb_rated[X_tmp_test[-3000][1]]

6

In [55]:
len(X_tmp_test)

66878

In [19]:
from collections import Counter

nb_ratings = Counter(dataset.anonymized.X[:, 0])
nb_rated = Counter(dataset.anonymized.X[:, 1])

In [20]:
nb_ratings.most_common(5)

[(1855, 3059), (1046, 2912), (93, 2483), (1306, 2151), (1502, 2017)]

In [385]:
PIG = 1046
print(list(X_train[:, 0]).count(PIG))
print(list(X_tmp_test[:, 0]).count(PIG))

2320
592


In [68]:
from scipy.sparse import coo_matrix

nb_users = dataset.anonymized.nb_users
nb_works = dataset.anonymized.nb_works
row = X_train[:, 0]
col = X_train[:, 1]
data = y_train
M = coo_matrix((data, (row, col)), shape=(nb_users, nb_works)).tocsr()

In [386]:
len(M[PIG].data)

2320

In [71]:
from scipy.sparse import coo_matrix

nb_users = dataset.anonymized.nb_users
nb_works = dataset.anonymized.nb_works
row = X_test[:, 0]
col = X_test[:, 1]
data = y_test
secret = coo_matrix((data, (row, col)), shape=(nb_users, nb_works)).tocsr()

In [387]:
len(secret[PIG].data)

592

In [506]:
def relu(x):
    return max(-1, min(1, x))

def ident(x):
    return x

In [507]:
Counter(values)

Counter({-1.1676416399158847: 1,
         -1.1491829435443137: 1,
         -0.84148292472221797: 1,
         -0.03462047782438793: 1,
         0.076723814485728292: 1,
         0.078985447331703318: 1,
         0.083188059369782685: 1,
         0.10461310060452478: 1,
         0.25036640623481432: 1,
         0.25076801234805934: 1,
         0.25731148325268571: 1,
         0.285844308136046: 1,
         0.30371543334555895: 1,
         0.31767885177443378: 1,
         0.36010121626691483: 1,
         0.47885700830795197: 1})

In [429]:
from sklearn.linear_model import Lasso, ElasticNet

reg = Lasso(alpha=0.001, normalize=True)
for user_id in [PIG]:
    i2 = []
    v2 = []
    indices = M[user_id].indices
    values = M[user_id].data
    for i, v in zip(indices, values):
        if v != -0.5:
            i2.append(i)
            v2.append(v)
    reg.fit(T[indices], values)

In [430]:
for user_id in [PIG]:
    indices = secret[user_id].indices
    y_pred_lr = list(map(ident, reg.predict(T[indices])))
    special = []
    y_special = []
    for work_id in indices:
        if nb_rated[work_id] < 3:
            special.append(work_id)
            y_special.append(secret[user_id, work_id])

In [442]:
y_als = als.predict(np.array([(PIG, work_id) for work_id in special]))

In [443]:
Counter(y_special)

Counter({-2.0: 8, -0.5: 46, 0.1: 1, 0.5: 4, 2.0: 17})

In [444]:
y_true = secret[user_id].data

In [445]:
Counter(y_true)

Counter({-2.0: 53, -0.5: 277, 0.1: 32, 0.5: 52, 2.0: 178})

In [446]:
#als.compute_rmse(y_als, y_true)
als.compute_rmse(y_als, y_special)

1.2279867191521727

In [447]:
y_als[:10]

array([ 0.01197922,  0.01197922,  0.01197922,  0.01197922,  0.01197922,
        1.5850158 ,  0.01197922,  0.01197922,  0.01197922,  0.64119385])

In [448]:
y_lr_special = reg.predict(T[special])

In [449]:
#als.compute_rmse(y_pred_lr, y_true)
als.compute_rmse(y_lr_special, y_special)

1.1685289466230184

In [450]:
y_special[:10]

[-0.5, -0.5, -2.0, -2.0, -0.5, 2.0, -0.5, -0.5, -0.5, 2.0]

In [451]:
y_lr_special[:10]

array([ 0.51698216, -0.09516446, -0.23657821,  0.38801507, -0.37281518,
        0.47601378, -0.0969073 , -0.0715639 ,  0.56659119,  0.39945745])

In [21]:
i_test_ordered = sorted(i_test, key=lambda i: nb_rated[dataset.anonymized.X[i][1]], reverse=True)

In [22]:
X_test_ordered = dataset.anonymized.X[i_test_ordered]
y_test_ordered = dataset.anonymized.y[i_test_ordered]

In [23]:
K = len(X_test_ordered)
X_tmp_test = X_test_ordered[-K:]
y_tmp_test = y_test_ordered[-K:]

In [43]:
y_pred_als = als.predict(X_tmp_test)
print(y_tmp_test[:5])
print(y_pred_als[:5])
print('ALS Test error', als.compute_rmse(y_pred_als, y_tmp_test))
#y_pred_xals = xals.predict(X_tmp_test)
#print('XALS Test error', xals.compute_rmse(y_pred_xals, y_tmp_test))
#print(y_pred_xals[:5])
y_delta_res = []
for user_id, work_id in X_tmp_test:
    y_delta_res.append(M_pred[user_id][work_id])
y_delta_res = np.array(y_delta_res)
y_new = y_pred_als + 2 * y_delta_res
print(y_new[:5])
print(y_delta_res[:5])
print('ALS+delta Test error', als.compute_rmse(y_new, y_tmp_test))

[ 2.   2.   0.5  2.   0.1]
[ 0.48589134  1.94460025  1.13536123  0.92859408  1.61266596]
ALS Test error 1.16744780835


NameError: name 'M_pred' is not defined

In [19]:
als = MangakiALS(R + nb_tags, LAMBDA=0.3)
als.NB_ITERATIONS = 20
als.set_parameters(nb_users, nb_works)
als.fit(X[i_train], y[i_train])
#print(xals.U.shape)
#print(xals.U)
y_pred = als.predict(X[i_train])
print('Train error', als.compute_rmse(y_pred, y[i_train]))
y_pred = als.predict(X[i_test])
print(y[i_test])
print(y_pred)
print('Test error', als.compute_rmse(y_pred, y[i_test]))

Computing M: (5 × 4)
Chrono: fill and center matrix [0q, 0ms]
Shapes (5, 13) (13, 4)
Chrono: factor matrix [0q, 13ms]
Train error 0.407541177275
[ 2.25842611  3.07814287  2.53238829  2.79468316]
[ 3.03586777  2.73180058  2.93662579  2.89181462]
Test error 0.473606111409


In [20]:
print(M_true)
print(als.means)
print((als.M.T + als.means).T)

[[ 3.07814287  2.99662165  2.08916146  2.82286951]
 [ 3.12601251  1.87590014  2.53238829  3.58227232]
 [ 2.83954362  2.23948638  2.58370636  2.56809017]
 [ 4.12795464  2.25842611  2.19946811  2.47225194]
 [ 2.43309646  2.52171932  2.79468316  3.34326677]]
[ 2.63621754  2.86139499  2.55770663  2.9332249   2.76602752]
[[ 2.73180058  2.70027044  2.54446537  2.56782888]
 [ 2.86027437  2.63843605  2.93662579  3.00918137]
 [ 2.60100665  2.5191739   2.53868454  2.57140797]
 [ 3.32771838  3.03586777  2.60851086  2.75794132]
 [ 2.64127143  2.66435256  2.89181462  2.86724358]]


In [21]:
xals = MangakiXALS(R, LAMBDA=0.2)
xals.NB_ITERATIONS = 20
xals.load_tags(T_true)
xals.set_parameters(nb_users, nb_works)
xals.fit(X[i_train], y[i_train])
#print(xals.U.shape)
#print(xals.U)
y_pred = xals.predict(X[i_train])
print('Train error', xals.compute_rmse(y_pred, y[i_train]))
y_pred = xals.predict(X[i_test])
#print(y[i_test])
#print(y_pred)
print('Test error', xals.compute_rmse(y_pred, y[i_test]))
print(M_true)
print('means', xals.means)
print((xals.M.T + xals.means).T)
print('V', xals.VT)

Computing M: (5 × 4)
Chrono: fill and center matrix [0q, 1ms]
shape X_train (16, 2)
Step 0: 3.2354964193036886
shape X_train (16, 2)
Step 1: 0.16132774944187103
shape X_train (16, 2)
Step 2: 0.12252481352540687
shape X_train (16, 2)
Step 3: 0.12123771284547814
shape X_train (16, 2)
Step 4: 0.1210336230758187
shape X_train (16, 2)
Step 5: 0.1209886276295622
shape X_train (16, 2)
Step 6: 0.12097541695061939
shape X_train (16, 2)
Step 7: 0.12097081622660022
shape X_train (16, 2)
Step 8: 0.1209690872404515
shape X_train (16, 2)
Step 9: 0.12096841796998646
shape X_train (16, 2)
Step 10: 0.120968156015428
shape X_train (16, 2)
Step 11: 0.12096805305291274
shape X_train (16, 2)
Step 12: 0.12096801251605684
shape X_train (16, 2)
Step 13: 0.12096799654576817
shape X_train (16, 2)
Step 14: 0.12096799025218884
shape X_train (16, 2)
Step 15: 0.1209679877717131
shape X_train (16, 2)
Step 16: 0.120967986794037
shape X_train (16, 2)
Step 17: 0.12096798640867859
shape X_train (16, 2)
Step 18: 0.120967

In [24]:
from mangaki.utils.lasso import MangakiLASSO

lasso = MangakiLASSO(10, LAMBDA=0.1)
lasso.load_tags(T)
lasso.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)

#lasso.fit(dataset.anonymized.X[i_train], dataset.anonymized.y[i_train])
lasso.all_errors(X_train, X_test, y_train, y_test)

0


/Users/jilljenn/code/mangaki/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
Train error 0.801058124267
352 not
104 not
521 not
2002 not
159 not
1584 not
1263 not
521 not
492 not
160 not
1460 not
491 not
1887 not
352 not
687 not
Test error 1.42858994289


In [49]:
from mangaki.utils.balse import MangakiBALSE

balse = MangakiBALSE(10, LAMBDA=0.1)
balse.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)

balse.fit(dataset.anonymized.X[i_train], dataset.anonymized.y[i_train])
y_pred = balse.predict(X_train)
print('Train error', balse.compute_rmse(y_pred, y_train))
y_pred = balse.predict(X_test)
print('Test error', balse.compute_rmse(y_pred, y_test))

Computing M: (2079 × 9979)
Chrono: fill and center matrix [0q, 1631ms]
Shapes (2079, 10) (10, 9979)
Chrono: factor matrix [0q, 6117ms]
0


/Users/jilljenn/code/mangaki/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
Train error 0.99026743656
352 not
104 not
521 not
2002 not
159 not
1584 not
1263 not
521 not
492 not
160 not
1460 not
491 not
1887 not
352 not
687 not
Test error 1.1648712679


In [25]:
y_als = als.predict(X_tmp_test)
y_lasso = lasso.predict(X_tmp_test)

104 not
160 not
687 not
1263 not
159 not
1887 not
492 not
352 not
521 not
1460 not
521 not
491 not
352 not
1584 not
2002 not


In [26]:
nb_r = list(map(lambda x: nb_rated[x[1]], X_tmp_test))

In [30]:
import numpy as np

X = np.column_stack((nb_r, y_als, y_lasso))

In [31]:
y = y_tmp_test

In [32]:
X.shape

(66878, 3)

In [33]:
y.shape

(66878,)

In [34]:
import tensorflow as tf

In [35]:
gamma = tf.Variable(tf.random_normal([1]), name='gamma')

In [46]:
e1 = np.array([1, 0, 0])
e2 = np.array([0, 1, 0])
e3 = np.array([0, 0, 1])
pred = tf.sigmoid(1*(X.dot(e1) - gamma)) * X.dot(e2) + (1 - tf.sigmoid(1*(X.dot(e1) - gamma))) * X.dot(e3)
loss = tf.reduce_mean(tf.square(y - pred)) ** 0.5
reg_loss = loss

optimizer = tf.train.AdamOptimizer(1)
train_step = optimizer.minimize(reg_loss, var_list=[gamma])

init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

In [47]:
for i in range(100):
    sess.run(train_step)
    print('loss', sess.run(reg_loss), 'var', gamma.eval())

loss 1.16212 var [ 1.71178198]
loss 1.16196 var [ 2.65714264]
loss 1.16199 var [ 3.42634296]
loss 1.16206 var [ 3.95010829]
loss 1.16211 var [ 4.25023127]
loss 1.16214 var [ 4.37002373]
loss 1.16213 var [ 4.34614706]
loss 1.16211 var [ 4.20701599]
loss 1.16207 var [ 3.97550392]
loss 1.16202 var [ 3.6722436]
loss 1.16198 var [ 3.31983399]
loss 1.16195 var [ 2.94791985]
loss 1.16196 var [ 2.59679484]
loss 1.16199 var [ 2.31410527]
loss 1.16201 var [ 2.14085078]
loss 1.16202 var [ 2.09492922]
loss 1.16201 var [ 2.16750526]
loss 1.16198 var [ 2.33164287]
loss 1.16196 var [ 2.55180216]
loss 1.16195 var [ 2.79022217]
loss 1.16196 var [ 3.01211548]
loss 1.16197 var [ 3.19045711]
loss 1.16198 var [ 3.3086493]
loss 1.16198 var [ 3.36008191]
loss 1.16198 var [ 3.34600186]
loss 1.16197 var [ 3.27352715]
loss 1.16196 var [ 3.15452909]
loss 1.16196 var [ 3.00516701]
loss 1.16195 var [ 2.84534335]
loss 1.16195 var [ 2.69715738]
loss 1.16196 var [ 2.58177137]
loss 1.16196 var [ 2.5151875]
loss 1.1619

In [48]:
np.mean(np.square(y_als - y)) ** 0.5

1.1639611046066964